In [82]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import torch
from time import time
import io
import re
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from datetime import datetime
import glob, os
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

In [83]:
DAT_DIR = "/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/tensors/"
train_d = dict()
val_d = dict()
test_d=dict()

for d_set in ("src", "labels", "segs", 'clss', "attn"):
    train_d[d_set] = torch.load(DAT_DIR + d_set+"_"+"train.pt")
    val_d[d_set] = torch.load(DAT_DIR + d_set+"_"+"validation.pt")
    test_d[d_set] = torch.load(DAT_DIR + d_set+"_"+"test.pt")

train_dataset = TensorDataset(train_d["src"], train_d["src"],train_d["labels"], train_d["segs"], 
                              train_d["clss"], train_d["attn"])
val_dataset = TensorDataset(val_d["src"], val_d["src"],val_d["labels"], val_d["segs"], 
                              val_d["clss"], val_d["attn"])
test_dataset = TensorDataset(test_d["src"], test_d["src"],test_d["labels"], test_d["segs"], 
                              test_d["clss"], test_d["attn"])

In [84]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 4

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [59]:
from transformers import BertModel, AdamW, BertConfig,BertTokenizer
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)



In [60]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (

In [88]:
for i, x in enumerate(train_dataloader):
    pass
